In [8]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pandas as pd
import requests

In [9]:
url = 'https://nyheder.tv2.dk/seneste'
driver = webdriver.Chrome()
driver.get(url)

In [10]:
cookie = driver.find_element(By.CSS_SELECTOR, '#onetrust-reject-all-handler')
cookie.click()

In [11]:
Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')
Vis_flere.click()

In [161]:
"""
for i in range(5):
    Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')
    Vis_flere.click()
    time.sleep(5)
"""

"\nfor i in range(5):\n    Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')\n    Vis_flere.click()\n    time.sleep(5)\n"

In [12]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

Headline = []
Category = []
Authors = []
Time = []
All_text = []
nummer=0

max_retries = 10
retries = 0

for i in tqdm(range(5)):
    retries = 0
    while retries < max_retries:
        try:
            one_column_lists = soup.find_all('ul', {'data-columns': '1'})[i]
            break
        except:
            retries = retries + 1
            time.sleep(1)
    
    if retries == max_retries:
        print(f"Max tries at: Number {nummer} and i: {i}")
    
    for one_column_list in one_column_lists:
        artikler = one_column_list.find_all(class_='tc_teaser')

        for artikel in artikler:
            nummer = nummer + 1

            Category.append(artikel.find('span', class_='tc_label tc_label--color-nyheder').text)
            Headline.append(artikel.find('a', class_='tc_teaser__link').get('aria-label'))

            href = artikel.find('a', class_='tc_teaser__link').get('href') #if link else None

            text = []
            if href:
                driver.execute_script(f'window.open("{href}", "_blank");')
                driver.switch_to.window(driver.window_handles[-1])

                new_html = driver.page_source
                new_soup = BeautifulSoup(new_html, 'lxml')

                #Category.append(new_soup.find('div', class_='tc_page__label').find('a', class_='tc_label tc_label--color-nyheder').text)
                #Headline.append(new_soup.find('header', class_="tc_page__header").find('h1').text)

                Authors.append(new_soup.find('div', class_="tc_byline").find('strong').text)
                Time.append(new_soup.find('time', class_='tc_timestamp').get('datetime'))

                article_body_div = new_soup.find('div', class_='tc_page__body', attrs={'data-adobe-context': 'article-body'})
                if article_body_div:
                    rich_content_paragraphs = article_body_div.find_all('p')

                    for paragraph in rich_content_paragraphs:
                        text.append(paragraph.text)

                All_text.append(text)

                driver.close()
                driver.switch_to.window(driver.window_handles[0])

            #print(nummer, Category[nummer-1], ':', Headline[nummer-1])
    Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')
    Vis_flere.click()
    #print(i)

 80%|████████  | 4/5 [02:51<00:57, 57.34s/it]

Max tries at: Number 79 and i: 4


100%|██████████| 5/5 [04:52<00:00, 58.40s/it]


In [13]:
print(Category[])
print(Authors[0])
print(Headline[0])
print(Time[0])
print(All_text[0])

Samfund
Ritzau
Bilka klar med hjemmelevering
2023-08-14T06:54:30.465Z
['Bilka vil levere varer til kunderne om natten for at få konceptet til at være økonomisk bæredygtigt.', 'Flere danske supermarkedskæder er blevet besejret af de hårde vilkår, der præger markedet for onlinesupermarkeder.', 'Men nu bliver der gjort et nyt forsøg på at overkomme vanskelighederne, når Salling Group tager hul på Bilkas bud på bæredygtig hjemmelevering.', 'Fra tirsdag bliver det muligt at få leveret varer direkte til hjemmet gennem konceptet "BilkaToGo".', 'Det oplyser Salling Group, der står bag dagligvarekæden, i en pressemeddelelse.', " - Bag rattet sidder Bilka's egne servicebude, som tager hul på ruten fra seks hypermarkeder i henholdsvis Skalborg, Tilst, Vejle, Odense, Hundige og Ishøj, skriver Salling Group.", 'Bilka har i forvejen et lignende koncept, men her skal kunderne selv hente varerne i butikken.', 'Det hjælper dog koncernen med at have logistikken på plads, inden der tages hul på hjemmelev

In [20]:
data = {
    'Category': Category,
    'Authors': Authors,
    'Headline': Headline,
    'Time': Time,
    'All_text': All_text
}

df = pd.DataFrame(data)
df.head(40)

,Category,Authors,Headline,Time,All_text
0,Samfund,Ritzau,Bilka klar med hjemmelevering,2023-08-14T06:54:30.465Z,[Bilka vil levere varer til kunderne om natten...
1,Politik,Camilla Stampe,Oppositionen i frontalangreb på Løkkes Kina-ku...,2023-08-14T06:48:53.195Z,"[SF, DF og Danmarksdemokraterne kalder Løkkes ..."
2,Lokalt,Jakob Olling,Fem drenge anholdt efter biljagt,2023-08-14T05:53:58.271Z,[Fem drenge på 15 og 16 år blev natten til man...
3,Krigen i Ukraine,Sebastian Juel Ebling,Russisk bombefly over Nordsøen sendte danske F...,2023-08-14T04:55:53.909Z,[]
4,Politik,Marie Jensen,Løkke rejser til Kina i morgen: - Vi fylder mi...,2023-08-14T04:42:25.931Z,"[- Vi skal engagere os, men vi skal ikke være ..."
5,Udland,Signe Marie Frost,"Nyt iransk lovforslag kaldes ""helt absurd""",2023-08-14T03:56:12.031Z,"[I øjeblikket er protesterne i Iran ""stort set..."
6,Krimi,Ritzau,30-årig anholdt for blufærdighedskrænkelse på ...,2023-08-14T03:54:29.401Z,[En 30-årig mand fremstilles mandag i grundlov...
7,Udland,Ritzau,Festival lægger sag an efter kys på scene,2023-08-14T03:44:20.610Z,[The 1975 er blevet sagsøgt for 18 millioner k...
8,Samfund,Ritzau,Undersøgelse finder stor mistrivsel blandt pen...,2023-08-14T03:33:51.237Z,[På baggrund af svar fra 14.000 seniorer konkl...
9,Samfund,Pelle Lykkebo Mørk,Tog-kaos forsinker regional- og S-tog,2023-08-14T03:13:02.247Z,[]


In [128]:
#df.to_csv('Data.csv', index=False)